In [1]:
import pandas as pd
import numpy as np
dfTemp=pd.read_csv(r'C:\Users\Jaesung Park\Desktop\FP\GroupProject\kc_house_data.csv',keep_default_na=False)

In [2]:
dfTemp.shape

(21613, 21)

In [3]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [4]:
dfTemp.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [5]:
data = dfTemp
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('id')
data.price = data.price.astype(int)
data.bathrooms = data.bathrooms.astype(int)
data.floors = data.floors.astype(int)
data["house_age"] = data["date"].dt.year - data['yr_built']
data['renovated'] = data['yr_renovated'].apply(lambda yr: 0 if yr == 0 else 1)

data=data.drop('date', axis=1)
data=data.drop('yr_renovated', axis=1)
data=data.drop('yr_built', axis=1)

dataset = data[data.columns.difference(['id', 'long'])]

In [6]:
len(dataset.columns)

18

In [7]:
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression, RidgeCV

from sklearn import metrics
%matplotlib inline

In [8]:
dataset.isnull().sum().sum()

0

In [9]:
pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000

In [10]:
#X =  dataset[dataset.columns.difference(['price'])]
X = dataset[['house_age','sqft_living','grade','condition', 'sqft_living15','bathrooms','view','sqft_basement','floors','waterfront','renovated','bedrooms']]
#X = dataset[['house_age','sqft_living','grade','bathrooms','view','sqft_basement','waterfront','renovated','floors','sqft_living15','sqft_lot15','bedrooms']]

Y = np.log(dataset['price'])

In [11]:
len(X.columns)

12

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [13]:
regressorLR = LinearRegression()  
alphas = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1,0.5, 1, 10, 15, 20, 30, 40, 100]
regressorR=RidgeCV(alphas=alphas, normalize=True)

regressorLR.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
coeff_df = pd.DataFrame(regressorLR.coef_, X.columns, columns=['Coefficient'])  
coeff_df

,Coefficient
house_age,0.005419
sqft_living,0.000119
grade,0.208527
condition,0.044343
sqft_living15,0.000103
bathrooms,0.055333
view,0.036872
sqft_basement,0.000083
floors,0.119621
waterfront,0.359362


In [15]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif.round(1)

,VIF Factor,features
0,1.9,house_age
1,27.9,sqft_living
2,68.5,grade
3,25.8,condition
4,26.2,sqft_living15
5,16.1,bathrooms
6,1.5,view
7,2.5,sqft_basement
8,15.7,floors
9,1.2,waterfront


In [16]:
y_pred = regressorLR.predict(X_test)

In [17]:
print('Data Shape:', dfTemp.shape)
print('Mean of SalePrice', np.mean(dataset['price']))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))

Data Shape: (21613, 21)
Mean of SalePrice 540182.1587933188
Mean Absolute Error: 0.24695179849614668
Mean Squared Error: 0.09616889213212183
Root Mean Squared Error: 0.3101110964350064
R2: 0.6671696545834743
